<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/california_housing/defiend_DNN_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip show tensorflow

Name: tensorflow
Version: 2.3.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: grpcio, tensorflow-estimator, protobuf, keras-preprocessing, termcolor, wrapt, scipy, absl-py, wheel, astunparse, six, h5py, numpy, opt-einsum, gast, google-pasta, tensorboard
Required-by: fancyimpute


In [2]:
%cd /content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
%ls

/content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
auto_keras_outputs/          defiend_ShallowNN_regressor.ipynb
autokeras_regressor.ipynb    fetch_data.ipynb
datasets/                    xgb_regressor.ipynb
defiend_DNN_regressor.ipynb


In [3]:
import cloudpickle, os, pprint

import pandas as pd
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
from tensorflow.keras import Model, Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

## データ読み込み

In [4]:
F_COLS = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
TARGET_COL = 'MedHouseVal'

data_path = 'datasets/origin'
train_csv = 'train.csv'
val_csv = 'val.csv'
test_csv = 'test.csv'

train_df= pd.read_csv(os.path.join(data_path, train_csv), header=0)
val_df = pd.read_csv(os.path.join(data_path, val_csv), header=0)
test_df = pd.read_csv(os.path.join(data_path, test_csv), header=0)

print('train shape:{}'.format(train_df.shape))
print('validation shape:{}'.format(val_df.shape))
print('test shape:{}'.format(test_df.shape))

display(train_df.head())

train shape:(13209, 9)
validation shape:(3303, 9)
test shape:(4128, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,4.4063,15.0,6.104430,1.003165,1237.0,3.914557,34.00,-117.50,1.285
1,3.7222,35.0,5.539326,1.030899,1152.0,3.235955,33.87,-117.96,1.875
2,1.1094,49.0,3.773585,1.144654,1448.0,4.553459,37.95,-121.28,0.525
3,3.3239,9.0,5.610955,1.092697,1691.0,2.375000,36.82,-119.85,0.913
4,5.9629,17.0,6.867133,1.097902,808.0,2.825175,38.58,-121.81,2.860


## Deep Neural Networkモデルを構築

In [5]:
def model_fn(input_shape, output_size):
  '''モデルの定義'''
  model = Sequential([
    layers.Dense(64, activation='relu', input_shape=[input_shape]),
    layers.Dropout(rate=0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(rate=0.3),
    layers.Dense(output_size)
  ])
  return model


model = model_fn(input_shape=len(F_COLS), output_size=1)

model.compile(loss='mse',
              optimizer=Adam(learning_rate=0.001),
              metrics=['mae', 'mse'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                576       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,689
Trainable params: 2,689
Non-trainable params: 0
_________________________________________________________________


In [6]:
%time
BATCH_SIZE = 256
EPOCHS = 100

es = EarlyStopping(patience=5)

model.fit(x=train_df[F_COLS].values, y=train_df[TARGET_COL].values,
          batch_size=BATCH_SIZE, epochs=EPOCHS,
          validation_data=(val_df[F_COLS].values, val_df[TARGET_COL].values),
          callbacks=[es]
          )

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.48 µs
Epoch 1/100
52/52 [==============================] - 0s 5ms/step - loss: 19134.4961 - mae: 81.5737 - mse: 19134.4961 - val_loss: 112.5563 - val_mae: 5.8384 - val_mse: 112.5563
Epoch 2/100
52/52 [==============================] - 0s 2ms/step - loss: 2486.6572 - mae: 27.5317 - mse: 2486.6572 - val_loss: 4.5938 - val_mae: 1.7731 - val_mse: 4.5938
Epoch 3/100
52/52 [==============================] - 0s 3ms/step - loss: 902.8755 - mae: 14.6792 - mse: 902.8755 - val_loss: 17.4217 - val_mae: 3.5469 - val_mse: 17.4217
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 529.0455 - mae: 9.7620 - mse: 529.0455 - val_loss: 6.9245 - val_mae: 2.2295 - val_mse: 6.9245
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 244.2318 - mae: 6.7462 - mse: 244.2318 - val_loss: 4.5424 - val_mae: 1.7274 - val_mse: 4.5424
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 180.2058 - ma

## 前処理したデータの読み込み
行った前処理
- 標準化

In [7]:
std_data_path = 'datasets/std'
std_train_csv = 'std_train.csv'
std_val_csv = 'std_val.csv'
std_test_csv = 'std_test.csv'

std_train_df= pd.read_csv(os.path.join(std_data_path, std_train_csv), header=0)
std_val_df = pd.read_csv(os.path.join(std_data_path, std_val_csv), header=0)
std_test_df = pd.read_csv(os.path.join(std_data_path, std_test_csv), header=0)

print('std_train shape:{}'.format(std_train_df.shape))
print('std_validation shape:{}'.format(std_val_df.shape))
print('std_test shape:{}'.format(std_test_df.shape))

display(std_train_df.head())

std_train shape:(13209, 9)
std_validation shape:(3303, 9)
std_test shape:(4128, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,0.297052,-1.079924,0.251551,-0.185450,-0.170175,0.119535,-0.760080,1.026143,-0.667466
1,-0.062875,0.505159,0.040977,-0.132852,-0.242531,0.024947,-0.821013,0.796661,-0.152404
2,-1.437551,1.614717,-0.616988,0.082885,0.009438,0.208590,1.091352,-0.859600,-1.330936
3,-0.272433,-1.555449,0.067668,-0.015652,0.216291,-0.095059,0.561702,-0.146211,-0.992217
4,1.116028,-0.921416,0.535755,-0.005780,-0.535361,-0.032310,1.386643,-1.124003,0.707487


In [8]:
# 標準化器も読み込んでおく
def load_pkl(path):
  '''pklファイルの読み込み'''
  with open(path, 'rb') as f:
    obj = cloudpickle.load(f)
  return obj


pkl_name = 'std_scaler.pkl'
std_scaler = load_pkl(os.path.join(std_data_path, pkl_name))
std_scaler

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


StandardScaler(copy=True, with_mean=True, with_std=True)

## 前処理済みデータに対してモデルを構築

In [9]:
std_model = model_fn(input_shape=len(F_COLS), output_size=1)

std_model.compile(loss='mse',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=['mae', 'mse'])

es = EarlyStopping(patience=3)

std_model.fit(x=std_train_df[F_COLS].values, y=std_train_df[TARGET_COL].values,
              batch_size=BATCH_SIZE, epochs=EPOCHS,
              validation_data=(std_val_df[F_COLS].values, std_val_df[TARGET_COL].values),
              callbacks=[es]
              )

Epoch 1/100
52/52 [==============================] - 0s 4ms/step - loss: 0.9786 - mae: 0.7543 - mse: 0.9786 - val_loss: 0.6113 - val_mae: 0.5982 - val_mse: 0.6113
Epoch 2/100
52/52 [==============================] - 0s 2ms/step - loss: 0.6220 - mae: 0.5924 - mse: 0.6220 - val_loss: 0.5007 - val_mae: 0.5181 - val_mse: 0.5007
Epoch 3/100
52/52 [==============================] - 0s 2ms/step - loss: 0.5794 - mae: 0.5552 - mse: 0.5794 - val_loss: 0.4628 - val_mae: 0.4964 - val_mse: 0.4628
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 0.5226 - mae: 0.5266 - mse: 0.5226 - val_loss: 0.4256 - val_mae: 0.4743 - val_mse: 0.4256
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 0.4708 - mae: 0.5041 - mse: 0.4708 - val_loss: 0.3971 - val_mae: 0.4598 - val_mse: 0.3971
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 0.4659 - mae: 0.4901 - mse: 0.4659 - val_loss: 0.3828 - val_mae: 0.4430 - val_mse: 0.3828
Epoch 7/100
52/52 [===

## モデルの評価

### 評価指標の定義

In [10]:
SK_METRICS=[r2_score, explained_variance_score, mean_squared_error]

def _calc_score(y_true, y_pred, sklearn_metric):
  '''sklearnの評価指標のスコアを計算'''
  score = sklearn_metric(y_true=y_true, y_pred=y_pred)
  return score


def evaluete_sk_metrics(y_true, y_pred, sklearn_metrics):
  '''sklearnの各種、評価関数を使用'''
  result = {}
  for sk_metric in sklearn_metrics:
    result[sk_metric.__name__] = _calc_score(y_true=y_true, y_pred=y_pred, sklearn_metric=sk_metric)
  return result

### 推論結果の取得

In [11]:
def predict_multiple_times(model, inputs, input_names):
  '''何度か推論'''
  pred = {}
  for input, name in zip(inputs, input_names):
    pred[name] = model.predict(input)
  return pred

In [12]:
%time
reg_inputs = [train_df[F_COLS].values, val_df[F_COLS], test_df[F_COLS].values]
reg_input_names = [train_csv, val_csv, test_csv]

std_reg_inputs = [std_train_df[F_COLS].values, std_val_df[F_COLS].values, std_test_df[F_COLS].values]
std_input_names = [std_train_csv, std_val_csv, std_test_csv]


reg_preds = predict_multiple_times(model=model, inputs=reg_inputs, input_names=reg_input_names)
std_reg_preds = predict_multiple_times(model=std_model, inputs=std_reg_inputs, input_names=std_input_names)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs


### 評価の実行

In [13]:
model_name = 'ShallowNN'
std_model_name = 'std_ShallowNN'

eval_scores = {model_name: {}, std_model_name: {}}

for csv_name, y_true in zip([train_csv, val_csv, test_csv],
                            [train_df[TARGET_COL].values, val_df[TARGET_COL].values, test_df[TARGET_COL].values]):
  eval_scores[model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                          y_pred=reg_preds[csv_name], sklearn_metrics=SK_METRICS)

for csv_name, y_true in zip([std_train_csv, std_val_csv, std_test_csv], 
                            [std_train_df[TARGET_COL].values, std_val_df[TARGET_COL].values, std_test_df[TARGET_COL].values]):
  eval_scores[std_model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                              y_pred=std_reg_preds[csv_name], sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'ShallowNN': {'test.csv': {'explained_variance_score': -0.19024090705998442,
                            'mean_squared_error': 2.8355035404206133,
                            'r2_score': -1.1081552206026752},
               'train.csv': {'explained_variance_score': -0.2190799709166884,
                             'mean_squared_error': 2.6967030020720757,
                             'r2_score': -1.0551720248170615},
               'val.csv': {'explained_variance_score': -0.14233020031470045,
                           'mean_squared_error': 2.8340194839280963,
                           'r2_score': -1.0414726191009631}},
 'std_ShallowNN': {'std_test.csv': {'explained_variance_score': 0.7379766870985052,
                                    'mean_squared_error': 0.2702549543637324,
                                    'r2_score': 0.7363480464688159},
                   'std_train.csv': {'explained_variance_score': 0.7481227894748291,
                                     'mean_squared_err

## モデルの予測値を標準化から復元し評価する場合

In [14]:
def inverse_scale_values(values, scaler):
  '''scalerを使って値の復元'''
  return scaler.inverse_transform(values)


inverse_scale_preds = {}
# 目的変数をモデルの予測値に置き換え
for df, csv_name, key in zip([std_train_df, std_val_df, std_test_df], [std_train_csv, std_val_csv, std_test_csv], 
                             [train_csv, val_csv, test_csv]):
  df[TARGET_COL] = std_reg_preds[csv_name]
  tmp_df = pd.DataFrame(inverse_scale_values(df.values, std_scaler), columns=F_COLS+[TARGET_COL])
  inverse_scale_preds[key] = tmp_df[TARGET_COL].values

inverse_scale_preds

{'test.csv': array([1.87008941, 2.50364486, 2.7981052 , ..., 1.36596458, 3.23513048,
        3.82070102]),
 'train.csv': array([1.49912021, 1.66854893, 0.87408859, ..., 1.31983923, 1.18044045,
        2.23785628]),
 'val.csv': array([1.29993827, 0.90050021, 1.27258826, ..., 0.99922724, 1.08113334,
        0.97867055])}

In [15]:
# 標準化された値を復元した場合の評価スコア
for csv_name, y_true in zip([train_csv, val_csv, test_csv],
                            [train_df[TARGET_COL].values, val_df[TARGET_COL].values, test_df[TARGET_COL].values]):
  eval_scores[std_model_name][csv_name] = evaluete_sk_metrics(y_true=y_true,
                                                              y_pred=inverse_scale_preds[csv_name],
                                                              sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'ShallowNN': {'test.csv': {'explained_variance_score': -0.19024090705998442,
                            'mean_squared_error': 2.8355035404206133,
                            'r2_score': -1.1081552206026752},
               'train.csv': {'explained_variance_score': -0.2190799709166884,
                             'mean_squared_error': 2.6967030020720757,
                             'r2_score': -1.0551720248170615},
               'val.csv': {'explained_variance_score': -0.14233020031470045,
                           'mean_squared_error': 2.8340194839280963,
                           'r2_score': -1.0414726191009631}},
 'std_ShallowNN': {'std_test.csv': {'explained_variance_score': 0.7379766870985052,
                                    'mean_squared_error': 0.2702549543637324,
                                    'r2_score': 0.7363480464688159},
                   'std_train.csv': {'explained_variance_score': 0.7481227894748291,
                                     'mean_squared_err